In [1]:
import asyncio
import random
from functools import wraps


In [6]:

def retry_when_error(max_retries=4, base_delay=1, jitter=0.1):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return await func(*args, **kwargs)
                except Exception as e:
                    if retries == max_retries - 1:
                        print(f"Failed after {retries + 1} retries")
                        raise
                    retries += 1
                    delay = (base_delay * 2 ** retries)  + (random.random() * jitter)
                    print(f"Retrying {retries}/{max_retries} with delay {delay:.3f} after error: {e}")
                    await asyncio.sleep(delay)
            return await func(*args, **kwargs)
        return wrapper
    return decorator


In [11]:
# test it out
@retry_when_error(jitter=60)
async def test(arg1, kwarg1=None):
    if random.random() < 0.5:
        raise Exception('test') 
    
    return f"with normal return {arg1} and {kwarg1}"

In [12]:
for i in range(10):
    print(f"trial {i} {await test('arg1', kwarg1='kwarg1')}")

Retrying 1/4 with delay 61.840 after error: test
trial 0 with normal return arg1 and kwarg1
Retrying 1/4 with delay 18.888 after error: test
trial 1 with normal return arg1 and kwarg1
trial 2 with normal return arg1 and kwarg1
trial 3 with normal return arg1 and kwarg1
trial 4 with normal return arg1 and kwarg1
Retrying 1/4 with delay 55.917 after error: test
trial 5 with normal return arg1 and kwarg1
Retrying 1/4 with delay 19.557 after error: test
trial 6 with normal return arg1 and kwarg1
Retrying 1/4 with delay 49.767 after error: test
Retrying 2/4 with delay 43.468 after error: test
trial 7 with normal return arg1 and kwarg1
Retrying 1/4 with delay 22.298 after error: test
trial 8 with normal return arg1 and kwarg1
trial 9 with normal return arg1 and kwarg1
